In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_dataset, load_dataset_with_lables
from data_parser import get_tags_and_labels, get_vocab, get_tags

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# Get dataframe with tags and category_id columns
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)

# Creating different coloumn based on different tags
new = tab_dataframe["tags"].str.split(".", expand = True)

max_of_tags = 15

for i in range(0, max_of_tags):
    name = "tags"+str(i)
    tab_dataframe[name] = new[i]

# Dropping old tags columns 
tab_dataframe.drop(columns =["tags"], inplace = True) 
tab_dataframe.fillna("notags", inplace = True) 


tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,10,Eminem,Walk,On,Water,Aftermath/Shady/Interscope,Rap,notags,notags,notags,notags,notags,notags,notags,notags,notags
1,23,plush,bad unboxing,unboxing,fan mail,idubbbztv,idubbbztv2,things,best,packages,plushies,chontent chop,notags,notags,notags,notags
2,23,racist superman,rudy,mancuso,king,bach,racist,superman,love,rudy mancuso poo bear black white official mus...,iphone x by pineapple,lelepons,hannahstocking,rudymancuso,inanna,anwar
3,24,ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,dares,no truth,comments,comedy,funny,stupid,fail,notags
4,10,edsheeran,ed sheeran,acoustic,live,cover,official,remix,official video,lyrics,session,notags,notags,notags,notags,notags


In [3]:
# Getting leaning vocab
vocab = get_vocab(get_tags(csvpath))
voc_di = {i:vo for i,vo in enumerate(vocab)}
voc_di[0] = 'notags'
voc_di = {vo:i for i, vo in voc_di.items()}

In [4]:
# Mapping tags to indexes in vocab
for index in range(0, max_of_tags):
    tab_dataframe['tags{}'.format(index)] = tab_dataframe['tags{}'.format(index)].map(voc_di)
    
tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,10,25063.0,32626.0,50982.0,36888.0,127176.0,96607.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,62947.0,45813.0,119613.0,85192.0,88271.0,104391.0,97286.0,115120.0,129107.0,108044.0,147369.0,0.0,0.0,0.0,0.0
2,23,109256.0,108170.0,17941.0,47206.0,136325.0,33780.0,151294.0,131860.0,140820.0,146360.0,78087.0,87933.0,11408.0,40193.0,42308.0
3,24,86521.0,111828.0,37882.0,69755.0,33744.0,86575.0,148814.0,86156.0,36579.0,119561.0,133140.0,121613.0,28415.0,126698.0,0.0
4,10,26666.0,72079.0,139817.0,24897.0,55936.0,100063.0,133803.0,77820.0,67729.0,130163.0,0.0,0.0,0.0,0.0,0.0


In [5]:
for column in tab_dataframe.columns:
    if column == 'category_id':
        pass
    else:
        if tab_dataframe[column].max() == 0:
            tab_dataframe[column] = tab_dataframe[column]

        else:
            tab_dataframe[column] = tab_dataframe[column] / tab_dataframe[column].max()

tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,10,0.165424,0.215349,0.336502,0.243474,0.839512,0.637649,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,23,0.415470,0.302390,0.789493,0.562297,0.582693,0.689027,0.642249,0.759898,0.852203,0.713105,0.972925,0.000000,0.000000,0.000000,0.000000
2,23,0.721124,0.713979,0.118418,0.311576,0.899906,0.222963,0.998792,0.870398,0.929517,0.965996,0.515528,0.580401,0.075302,0.265299,0.279303
3,24,0.571066,0.738124,0.250036,0.460408,0.222750,0.571433,0.982420,0.568709,0.241449,0.789119,0.878986,0.802705,0.187563,0.836285,0.000000
4,10,0.176004,0.475760,0.922848,0.164329,0.369244,0.660460,0.883322,0.513684,0.447062,0.859094,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
# Create train,validation and test examples from dataframe
# train_ds, val_ds, test_ds = load_dataset(tab_dataframe)
train_ds, test_ds, train_lb, test_lb = load_dataset_with_lables(tab_dataframe, 'category_id')


In [15]:
# first 5 training examples
print(train_ds.values[0:5])
train_lb.values[0:5]

[[0.25190749 0.9352224  0.80144681 0.07559387 0.16996066 0.76621894
  0.05683371 0.92398379 0.81817582 0.16576245 0.07678748 0.30261247
  0.92399139 0.5160098  0.21040158]
 [0.12945191 0.53768572 0.38265151 0.85247546 0.03099916 0.22584073
  0.98334401 0.97767568 0.37380691 0.34835525 0.94979864 0.366855
  0.         0.         0.        ]
 [0.62139293 0.03698937 0.30978971 0.77382563 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.11689812 0.47292793 0.57796391 0.34187199 0.84584918 0.43597241
  0.78427088 0.99390735 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.51459989 0.31613235 0.44297256 0.53513039 0.34642348 0.97360483
  0.63603715 0.69103727 0.8480904  0.04633296 0.18647917 0.69089265
  0.23209194 0.6873552  0.        ]]


array([25, 10, 10, 22, 24], dtype=int64)

In [16]:
hidden = 32
model = tf.keras.Sequential([
    layers.Flatten(),
    layers.Dense(hidden,activation='relu', input_shape=(1,15)),
    layers.Dropout(0.7),
    layers.Dense(2*hidden,activation='relu', input_shape=(1, 64)),
    layers.BatchNormalization(),
    layers.Dropout(0.7),
    layers.Dense(44, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.01), input_shape=(1, 64))
])

model.compile(optimizer='nadam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'binary_crossentropy'],
              )

history = model.fit(train_ds.values, train_lb.values,
          batch_size=100,
          epochs=200)


W0807 12:05:50.909368 18684 nn_ops.py:4220] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0807 12:05:51.081476 18684 nn_ops.py:4220] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Train on 32704 samples
Epoch 1/200
32704/32704 [==============================] - 2s 60us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 2/200
32704/32704 [==============================] - 1s 31us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 3/200
32704/32704 [==============================] - 1s 31us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 4/200
32704/32704 [==============================] - 1s 31us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 5/200
32704/32704 [==============================] - 1s 34us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 6/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 7/200
32704/32704 [==============================] - 1s 31us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 8/200
32704/32704 [

32704/32704 [==============================] - 1s 33us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 61/200
32704/32704 [==============================] - 1s 33us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 62/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 63/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 64/200
32704/32704 [==============================] - 1s 33us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 65/200
32704/32704 [==============================] - 1s 34us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 66/200
32704/32704 [==============================] - 1s 34us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 67/200
32704/32704 [============================

32704/32704 [==============================] - 1s 34us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 120/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 121/200
32704/32704 [==============================] - 1s 33us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 122/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 123/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 124/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 125/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 126/200
32704/32704 [=====================

Epoch 178/200
32704/32704 [==============================] - 1s 34us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 179/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 180/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 181/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 182/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 183/200
32704/32704 [==============================] - 1s 32us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 184/200
32704/32704 [==============================] - 1s 33us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 185/200
32704/32704 [=======